In [1]:
# mlflow test

import mlflow
import numpy as np
import pandas as pd

In [5]:
mlflow.set_tracking_uri("http://ec2-54-198-108-56.compute-1.amazonaws.com:5000")

with mlflow.start_run():
    mlflow.log_param("param1", 15)
    mlflow.log_metric("metric1", 0.89)


🏃 View run melodic-moth-102 at: http://ec2-54-198-108-56.compute-1.amazonaws.com:5000/#/experiments/0/runs/f4039bb8f9544e3eac3ebc5be056a6bb
🧪 View experiment at: http://ec2-54-198-108-56.compute-1.amazonaws.com:5000/#/experiments/0
